In [1]:
# import os
# import shutil
# adjectives_folder = "adjectives"
# for folder_name in os.listdir(adjectives_folder):
#     folder_path = os.path.join(adjectives_folder, folder_name)
#     if os.path.isdir(folder_path):
#         for subfolder_name in os.listdir(folder_path):
#             subfolder_path = os.path.join(folder_path, subfolder_name)
#             if os.path.isdir(subfolder_path) and subfolder_name.lower() =='extra':
#                 print(f"Removing extra folder: {subfolder_path}")
#                 shutil.rmtree(subfolder_path)

In [43]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,Bidirectional
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import mediapipe as mp

In [3]:
# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

I0000 00:00:1726403061.828006  449485 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1


In [4]:
def extract_landmarks(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(rgb_frame)
    
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([lh, rh, pose])

In [5]:
# def process_video(video_path, max_frames=60):
#     cap = cv2.VideoCapture(video_path)
#     frames = []
#     while len(frames) < max_frames and cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break
#         landmarks = extract_landmarks(frame)
#         frames.append(landmarks)
#     cap.release()

#     if len(frames) < max_frames:
#         frames = frames + [np.zeros_like(frames[0])] * (max_frames - len(frames))
#     elif len(frames) > max_frames:
#         frames = frames[:max_frames]
    
#     return np.array(frames)

In [18]:
def process_video(video_path, max_frames=90):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < max_frames and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        landmarks = extract_landmarks(frame)
        frames.append(landmarks)
    cap.release()
    if len(frames) < max_frames:
        frames = frames + [np.zeros_like(frames[0])] * (max_frames - len(frames))
    elif len(frames) > max_frames:
        frames = frames[:max_frames]
    return np.array(frames)

In [7]:
def load_dataset(data_path, max_frames=90):
    X, y = [], []
    for gesture_id, gesture_folder in enumerate(sorted(os.listdir(data_path))):
        gesture_path = os.path.join(data_path, gesture_folder)
        for video_file in os.listdir(gesture_path):
            video_path = os.path.join(gesture_path, video_file)
            processed_video = process_video(video_path, max_frames)
            X.append(processed_video)
            y.append(gesture_id)
    return np.array(X), np.array(y)

In [135]:
def build_model(input_shape, num_classes):
    model = Sequential([
        LSTM(64, return_sequences=True, activation='tanh', input_shape=input_shape),
        LSTM(128, return_sequences=True, activation='tanh'),
        LSTM(64, return_sequences=True, activation='tanh'),
        LSTM(64, return_sequences=False, activation='tanh'),
        Dense(64, activation='tanh'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [9]:
data_path = 'adjectives/'
X, y = load_dataset(data_path)
y=to_categorical(y)

W0000 00:00:1726403061.940695  449859 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726403061.950221  449862 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726403061.952182  449864 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726403061.952181  449859 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726403061.952486  449862 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726403061.956147  449861 inference_feedback_manager.cc:114] Feedback manager 

In [141]:
X.shape

(788, 90, 225)

In [114]:
X_full=X
y_full=y

In [115]:
y_full

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [137]:
input_shape = (X.shape[1], X.shape[2]) 
num_classes = 59

In [138]:
model = build_model(input_shape, num_classes)

In [139]:
history = model.fit(X, y, epochs=500, batch_size=64,validation_split=0.2)
test_loss, test_acc = model.evaluate(X_test, y_test)
model.save('sign_language_model_demo.h5')

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 209ms/step - accuracy: 0.0318 - loss: 4.0682 - val_accuracy: 0.0084 - val_loss: 4.1837
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 184ms/step - accuracy: 0.0271 - loss: 4.0018 - val_accuracy: 0.0000e+00 - val_loss: 4.4353
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 179ms/step - accuracy: 0.0602 - loss: 3.8818 - val_accuracy: 0.0000e+00 - val_loss: 4.8500
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 184ms/step - accuracy: 0.0546 - loss: 3.8408 - val_accuracy: 0.0000e+00 - val_loss: 5.0547
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - accuracy: 0.0434 - loss: 3.7643 - val_accuracy: 0.0000e+00 - val_loss: 5.3705
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - accuracy: 0.0516 - loss: 3.6978 - val_accuracy: 0.0000e+00 - val_loss: 5.6315
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 196ms/step - accuracy: 0.0756 - loss: 3.6769 - val_accuracy: 0.0000e+00 - val_loss: 5.8410
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 199ms/step - accuracy: 0.0627 - loss: 3.6247 - 

In [3]:
#sign_language_model=>accuracy=71%   max_frame=60
#sign_language_model_test=>accuracy=57%  max_frame=90
#sign_language_model_demo=>accuracy=66%  max_frame=90

In [2]:
print(f"Test accuracy: {test_acc}")

NameError: name 'test_acc' is not defined